<a href="https://colab.research.google.com/github/Krishnaugale353/Upgrad-Capstone/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

jhb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('/content/drive/MyDrive/Final Capstone Project/Intensity.csv')

In [3]:
# Check the shape of the combined dataset
print("Dataset dimensions:", data.shape)


Dataset dimensions: (2039, 2)


In [4]:
# Display the first 5 rows of the combined dataset
data.head()


content  intensity
0  Wants to know how the hell I can remember word...  happiness
1  Love is a long sweet dream & marriage is an al...  happiness
2  The world could be amazing when you are slight...  happiness
3  My secret talent is getting tired without doin...  happiness
4  Khatarnaak Whatsapp Status Ever… Can\’t talk, ...  happiness

In [5]:
# Check for any missing values in the dataset
data.isnull().sum()


content      0
intensity    0
dtype: int64

In [6]:
# Count the number of each label in the 'intensity' column
data['intensity'].value_counts()


intensity
happiness    708
angriness    696
sadness      635
Name: count, dtype: int64

In [7]:
# Load NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
data.columns

Index(['content', 'intensity'], dtype='object')

In [9]:


# Create a lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean the text while retaining emojis
def clean_text(text):
    # Lowercase the text
    text = text.lower()
    # Retain emojis, remove special characters and numbers
    text = re.sub(r'[^a-z\s\U0001F600-\U0001F64F]', '', text)  # Retains emojis in the range
    # Tokenization
    words = nltk.word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join the words back into a single string
    return ' '.join(words)

# Apply the cleaning function to the 'content' column
data['cleaned_content'] = data['content'].apply(clean_text)
data.drop(columns=['content'], inplace=True)

# Preview the cleaned data
data.head()

intensity                                    cleaned_content
0  happiness  want know hell remember word song year ago can...
1  happiness         love long sweet dream marriage alarm clock
2  happiness               world could amazing slightly strange
3  happiness     secret talent getting tired without anything 😀
4  happiness  khatarnaak whatsapp status ever cant talk wife...

In [10]:
duplicate_count = data.duplicated().sum()
print("Number of duplicate entries:", duplicate_count)

Number of duplicate entries: 545


In [11]:
if duplicate_count > 0:
    duplicates = data[data.duplicated()]
    print("Duplicate entries:")
    print(duplicates)

Duplicate entries:
      intensity                                    cleaned_content
150   happiness  life give hundred reason cry show life thousan...
157   happiness  next time think wonderful beautiful thing life...
243   happiness                           birthday quote boyfriend
402   happiness                                    real smile come
421   happiness                        love sweet new sweeter true
...         ...                                                ...
2002    sadness  hurt still love true love isnt found built sil...
2003    sadness  hurt still love true love isnt found built sil...
2004    sadness  hurt still love true love isnt found built sil...
2005    sadness  hurt still love true love isnt found built sil...
2024    sadness  dont waste tear cry someone doesnt even deserv...

[545 rows x 2 columns]


In [12]:
# Remove duplicate entries
data = data.drop_duplicates()

# Check the new dimensions of the dataset
print("New dataset dimensions:", data.shape)


New dataset dimensions: (1494, 2)


In [13]:
# Count the number of each label in the 'intensity' column
data['intensity'].value_counts()


intensity
happiness    697
angriness    412
sadness      385
Name: count, dtype: int64

In [19]:


# Data preparation: Tokenize the cleaned content
sentences = [sentence.split() for sentence in data['cleaned_content']]  # Tokenize each sentence into words

# Train a Word2Vec model using CBOW (sg=0) or Skip-gram (sg=1)
word2vec_model = Word2Vec(
    sentences,
    vector_size=300,
    window=5,
    sg=1,
    min_count=2,
    workers=4
)

# Save the model if needed
word2vec_model.save("word2vec_model.model")

# Create a function to get the average embedding for each sentence
def get_sentence_embedding(sentence, model, embedding_dim=300):
    words = sentence.split()
    embedding_matrix = [model.wv[word] for word in words if word in model.wv]

    if embedding_matrix:
        return np.mean(embedding_matrix, axis=0)
    else:
        return np.zeros(embedding_dim)

# Generate sentence embeddings using the trained model
X_embeddings = np.array([get_sentence_embedding(sentence, word2vec_model, 300) for sentence in data['cleaned_content']])
y = data['intensity']


In [20]:
# Apply SMOTE to balance the classes after creating embeddings
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_embeddings, y)

# Check the new class distribution
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

Class distribution after SMOTE:
intensity
happiness    697
angriness    697
sadness      697
Name: count, dtype: int64


In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
